In [ ]:
!pip install deepface

In [ ]:
from deepface import DeepFace

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


### Take any couple of images and compare them in all cases (beautified, real, modified)

In [ ]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import random

from PIL import Image, ImageFilter

In [ ]:
def rescale(data):
    return ((np.array(data) - np.min(data)) / (np.max(data) - np.min(data))).tolist()

In [ ]:
df_dict = {}

In [ ]:
original_folder_b = #this is your path to FairBeauty (to make it easier, move all the images in the subfolders to the main dir)
original_folder = #this is your path to FairFaces

In [ ]:
!mkdir /content/low_resolution
!mkdir /content/low_resolution_2

models = ["VGG-Face", "Facenet", "DeepFace"]

for model in models:

  img_names = os.listdir(original_folder_b)

  results_b = []
  results = []
  results_l = []
  results_l2 = []

  while (len(results_b)<500):
    n1, n2 = random.sample(img_names, 2)
    img_names.remove(n1)
    img_names.remove(n2)

    path1_b = original_folder_b+n1
    path2_b = original_folder_b+n2

    try:
      result_b = DeepFace.verify(img1_path = path1_b, img2_path = path2_b, distance_metric="euclidean", model_name=model, enforce_detection=False)
    except ValueError as e:
      print(e)
      continue

    path1 = original_folder+n1
    path2 = original_folder+n2
    
    try:
      result = DeepFace.verify(img1_path = path1, img2_path = path2, distance_metric="euclidean", model_name=model, enforce_detection=False)
    except ValueError as e:
      print(e)
      continue

    img1 = Image.open(path1)
    low_path1 = "./low_resolution/0"+n1.split('.')[0]+".jpg"
    img1.filter(ImageFilter.GaussianBlur(2)).save(low_path1)

    img2 = Image.open(path2)
    low_path2 = "./low_resolution/0"+n2.split('.')[0]+".jpg"
    img2.filter(ImageFilter.GaussianBlur(2)).save(low_path2)

    try:
      result_l = DeepFace.verify(img1_path = low_path1, img2_path = low_path2, distance_metric="euclidean", model_name=model, enforce_detection=False)
    except ValueError as e:
      print(e)
      continue

    low_path3 = "/content/low_resolution_2/0"+n1.split('.')[0]+".jpg"
    img1.filter(ImageFilter.GaussianBlur(3)).save(low_path3)
    #img1.resize((64, 64), resample=Image.BILINEAR).resize(img1.size,Image.NEAREST).save(low_path3)

    low_path4 = "/content/low_resolution_2/0"+n2.split('.')[0]+".jpg"
    img2.filter(ImageFilter.GaussianBlur(3)).save(low_path4)
    #img2.resize((64, 64), resample=Image.BILINEAR).resize(img2.size,Image.NEAREST).save(low_path4)

    try:
      result_l2 = DeepFace.verify(img1_path = low_path3, img2_path = low_path4, distance_metric="euclidean_l2", model_name=model, enforce_detection=False)
    except ValueError as e:
      print(e)
      continue

    results_b.append(result_b["distance"])
    results.append(result["distance"])
    results_l.append(result_l["distance"])
    results_l2.append(result_l2["distance"])

  df_dict[model] = rescale(results_b + results + results_l + results_l2)

In [ ]:
type_b = ["beauty"] * len(results_b)
type_o = ["original"] * len(results_b)
type_l = ["gaussian2"] * len(results_b)
type_l2 = ["gaussian3"] * len(results_b)
df_dict["type"] = type_b + type_o + type_l + type_l2

In [ ]:
df = pd.DataFrame.from_dict(df_dict)

In [ ]:
df